# Preprocesamiento de datos (Tweets)

En este notebook se realiza el preprocesamiento de los datos que contienen los tweets para el entrenamiento de los modelos de aprendizaje automático extraidos de HuggingFace. Los datos han dido extraídos de la plataforma de Kaggle, en el siguiente [enlace](https://www.kaggle.com/datasets/alishafaghi/hashtag-tesla-tweets). El dataset contiene más de 150.000 tweets etiquetados con #tesla.

El objetivo es obtener un dataset con una única columna de tweets en inglés, para su posterior etiquetado por parte del alumno. El etiquetado de datos será discutido en la memoria, y realizado en los scripts de la carpeta `labelling`.

In [1]:
import pandas as pd
import yfinance as yf
from langdetect import detect
import preprocessor as p
import string
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import nltk
from nltk.corpus import stopwords


nltk.download('stopwords')
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\34644\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocesamiento del dataset de tweets

### Overview

En esta sección se realiza la carga de datos y análisis de alto nivel del datafrfame

In [5]:
df_tweets = pd.read_csv('../../data/raw/tesla_tweets.csv')
df_tweets.sample(5)

,Date & Time,Profile Picture Link,Twitter ID,Tweet Text,Tweet Link
21236,"May 13, 2022 at 06:16AM",http://pbs.twimg.com/profile_images/152205270...,@Benoitclair,Quand tu découvres que Buzz #Aldrin ⁦@TheRealB...,https://twitter.com/Benoitclair/status/1524928...
113650,"September 19, 2022 at 07:13PM",http://pbs.twimg.com/profile_images/143372654...,@XeronBot,RT @VeilleCyber3: An #AI used #medical #notes ...,https://twitter.com/XeronBot/status/1571872527...
118741,"September 26, 2022 at 08:08PM",http://pbs.twimg.com/profile_images/121896452...,@TeslaradarB,Ahoy! Taurec just spotted a 2022 Tesla Model Y...,https://twitter.com/TeslaradarB/status/1574438...
129744,"October 12, 2022 at 06:12AM",http://pbs.twimg.com/profile_images/138533407...,@ESGESG1001,RT @TeslaAndDoge: I don’t get it. How is #Amaz...,https://twitter.com/ESGESG1001/status/15800260...
50673,"June 23, 2022 at 09:11AM",http://pbs.twimg.com/profile_images/153778715...,@nimanaamvar,@LeafsOfTea @SteveHakaStumpy @ValueAnalyst1 It...,https://twitter.com/nimanaamvar/status/1539830...


Como lo único que nos interesa es la columna de texto y su id, se descartan las demás columnas y se establece el tweet de cada post como índice del dataframe.

In [6]:

selected_columns = ['Tweet Link','Date & Time','Tweet Text']
df_tweets = df_tweets[selected_columns]
df_tweets.rename(columns={'Tweet Text':'Text', 'Tweet Link' : 'Link', 'Date & Time' : 'Date'}, inplace=True)
df_tweets.sample(5)

,Link,Date,Text
66109,https://twitter.com/socialstartnow1/status/154...,"July 14, 2022 at 09:16PM",$SINC News https://t.co/6OFldXHLk2 #wsj #nytim...
48227,https://twitter.com/saryosal/status/1538604668...,"June 19, 2022 at 11:58PM",RT @tribeofthought: I can't think of anyone wh...
44186,https://twitter.com/NowMyNews/status/153655378...,"June 14, 2022 at 08:09AM",Tesla fires Singapore executives for a year af...
19910,https://twitter.com/SellTesla/status/152412970...,"May 11, 2022 at 01:20AM",Just Sold: Model 3 Dec 2020 Performance Blue w...
108030,https://twitter.com/TeslaradarB/status/1569003...,"September 11, 2022 at 09:14PM",Swell! Leon was first to spot a 2019 Tesla Mod...


In [7]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Link    152000 non-null  object
 1   Date    152000 non-null  object
 2   Text    152000 non-null  object
dtypes: object(3)
memory usage: 3.5+ MB


Comprobamos que la columna de fecha no está en formato datetime, por lo que se convierte a dicho formato. 

In [8]:
df_tweets['Date'] = pd.to_datetime(df_tweets['Date'], format='mixed') # dejamos que pandas infiera el formato de fecha con el formato mixed
print(df_tweets.info())
df_tweets.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152000 entries, 0 to 151999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   Link    152000 non-null  object        
 1   Date    152000 non-null  datetime64[ns]
 2   Text    152000 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.5+ MB
None


,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,"RT @invest_answers: Crypto news, #Bitcoin Whal..."
1,https://twitter.com/JotaGe2014/status/15131737...,2022-04-10 19:45:00,#Tesla tiene récord de autos vendidos. Es impr...
2,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...
3,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,RT @CottonCodes: 🐒 #love in my #MariaCallas I ...
4,https://twitter.com/agseh/status/1513173864829...,2022-04-10 19:45:00,RT @RupiReportero_: 🙆‍♂️🚘 Al que le robaron la...


### Limpieza de datos

Se comprueba si existen valores nulos o atípicos, además de comprobar si existen tweets duplicados.


In [9]:
df_tweets.isnull().sum()

Link    0
Date    0
Text    0
dtype: int64

Al ser la fecha de publicación del tweet y el propio tweet en cuestion variables de tipos distintos, se hace su análisis por separado.

En la descripción del dataset en la página de Kaggle se indica que los tweets han sido extraídos entre Abril de 2022 y Noviembre de 2022. Por lo que se comprueba que no existen tweets con fecha anterior al 1 de enero de 2020. Comprobamos que no existen tweets con fechas posteriores ni anteriorers a las indicadas.

In [10]:
# Análisis de valores atípicos en la columna de fecha

(df_tweets['Date'].min(),
df_tweets['Date'].max())

(Timestamp('2022-04-10 19:44:00'), Timestamp('2022-11-12 15:11:00'))

### Preparación de la columna de text

En esta sección se realiza la limpieza de la columna de texto.Se eliminan los caracteres especiales, las menciones, los enlaces y los emojis utilizando la librería [tweet preprocessor](https://pypi.org/project/tweet-preprocessor/). Se convierte todo el texto a minúsculas, y seleccionamos sólo los tweets en inglés.	

Sería interesante poder elegir otros idiomas aparte del inglés para el etiquetado de datos, pero por simplicidad se ha decidido utilizar sólo el inglés, ya que es el idioma más utilizado en la plataforma de Twitter. Sería posible utilizar modelos preentrenados para la clasificación de sentimientos en diferentes idiomas, pero se ha decidido utilizar sólo el inglés para simplificar el proceso de etiquetado.

In [11]:
#dataframe vacío para almacenar los tweets que son en inglés
i = 0
df_aux = pd.DataFrame(columns=['Link','Date','Text'])
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY, p.OPT.NUMBER)
start_time = time.time()

for index, row in df_tweets.iterrows():
    cleaned_text = p.clean(row['Text'])  # Limpia el texto y guarda el resultado

    # Esta celda tarda mucho, por lo que si se desea saber el progreso se puede imprimir el porcentaje de tweets procesados
    if i % (len(df_tweets.index)/10) == 0:
        print('Progreso:',i,'/',len(df_tweets.index),'=',i/len(df_tweets.index)*100,'%')
        print('Tiempo transcurrido:',time.time()-start_time,'segundos')
    i+=1
    
    try:
        if detect(cleaned_text) == 'en': # Si el texto está en inglés, lo guardamos en el dataframe auxiliar
            df_aux.loc[len(df_aux.index)] = [row['Link'], row['Date'], cleaned_text]
    except:
        pass


Progreso: 0 / 152000 = 0.0 %
Tiempo transcurrido: 0.5858159065246582 segundos


In [ ]:
def eliminar_puntuacion(text):
    return text.translate(str.maketrans('', '', string.punctuation))

def eliminar_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

stop_words = set(stopwords.words('english'))

df_aux['Text'] = df_aux['Text'].apply(eliminar_puntuacion)
df_aux['Text'] = df_aux['Text'].apply(eliminar_stopwords)

print(len(df_aux.index))
df_aux.head(5)

124705


,Link,Date,Text
0,https://twitter.com/Jessica1988kk/status/15131...,2022-04-10 19:44:00,Crypto news Bitcoin Whales flying motorbikes ...
1,https://twitter.com/MmeCallas/status/151317374...,2022-04-10 19:45:00,love in my MariaCallas I KNOW Y art HOLDonLIN...
2,https://twitter.com/BotSecx/status/15131737626...,2022-04-10 19:45:00,love in my MariaCallas I KNOW Y art HOLDonLIN...
3,https://twitter.com/ElTendies/status/151317393...,2022-04-10 19:45:00,Tesla A Trillion Dollar Company Worlds Large...
4,https://twitter.com/LauraCory2013/status/15131...,2022-04-10 19:45:00,few chargingstations in my area I dont have a...


Como también fue notado en la caracterización de los datos, existen días que los mercados no están abiertos y por tanto no hay observaciones de las acciones. La estrategia de entrenamiento de los modelos de aprendizaje automático será la de predecir el precio de las acciones al día siguiente, por lo que se decide eliminar los tweets de los días en los que no hay observaciones de las acciones.

In [ ]:
df_stocks = pd.read_csv('../../data/raw/tesla_stocks.csv')
df_stocks['Date'] = pd.to_datetime(df_stocks['Date'], format='%Y-%m-%d')

df_copy = df_aux.copy()

"""
for date in df_copy['Date']:
    if date not in df_stocks['Date'].values:
        df_copy = df_copy[df_aux.Date != date]
""" 

mask = df_copy['Date'].isin(df_stocks['Date'])
df_copy = df_copy[mask]

print(len(df_copy.index))


In [ ]:
df_clean = df_aux.sample(1000)
df_clean.to_csv('../data/tesla_tweets_clean_random.csv', index=False)
df_aux.to_csv('../data/tesla_tweets_clean.csv', index=False)